In [ ]:
from faker import Faker
fake = Faker('pt_BR')
import datetime, json, requests, random
from dateutil.relativedelta import relativedelta
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey


HOST = 'https://padariacasedb.documents.azure.com:443/'
MASTER_KEY = 'KPxz6QHRsC0rnSC0A2Wi4C2PNVapVy0JU9GpCyUC4LRrD9Xqrmh18BCShyvXTHOrUxyNIFEf848tHGmv221eGA=='
DATABASE_ID = 'padariadb'

In [ ]:
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
db = client.create_database_if_not_exists(id=DATABASE_ID)
container_products = db.create_container_if_not_exists(
    id='products', 
    partition_key=PartitionKey(path="/productId"),
    offer_throughput=400
)

In [ ]:
items_providers = list(container_products.query_items(
    query="SELECT top 1 * FROM products p where RAND() <= @id",
    parameters=[
        { "name":"@id", "value": .3 }
    ],
    enable_cross_partition_query=True
))
for item in items_providers:
    print(item.get("productId"))
    print (item.get('summary'))
    item.get('summary')['lastQuantity'] = item.get('summary')['lastQuantity'] + 1
    #container_provider.upsert_item(body=item)

In [ ]:
items_providers = list(container_products.query_items(
    query="SELECT top 100 * FROM products p where RAND() <= @id ORDER BY p.id desc",
    parameters=[
        { "name":"@id", "value": .3 }
    ],
    enable_cross_partition_query=True
))
for item in items_providers:
    qtd = random.randint(10,100)
    price = round(random.random() * 1000,2)
    Product = {
        "id": fake.hexify(text='^^^^-^^^^-^^^^-^^^^'),
        "productId": item.get("productId"),
        "branchId": fake.random_number(3),
        "entryDate": datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
        "unit": "UN",
        "quantity": qtd,
        "price": price
        }
    item.get('summary')['lastQuantity'] = item.get('summary')['lastQuantity'] + qtd
    item.get('summary')['lastPrice'] = price
    item.get('summary')['laststockEntry'] = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    container_products.upsert_item(body=item)
    print (Product)